In [1]:
#!pip install langchain
#!pip install langchain_community
#!pip install pdfminer.six
#!pip install pillow_heif
#!pip install opencv-python
#!pip install pdf2image
#!pip install unstructured_inference
#!pip install pytesseract
#!pip install pikepdf
#!pip install unstructured_pytesseract
#!pip install langchain-text-splitters
#!pip install chromadb
#!pip install sentence-transformers

In [1]:
import os
import numpy as np
os.environ['OCR_AGENT'] = 'unstructured.partition.utils.ocr_models.tesseract_ocr.OCRAgentTesseract'
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredPDFLoader,UnstructuredFileLoader,OnlinePDFLoader
from pdfminer.pdfparser import PDFSyntaxError
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [2]:
# Define the loader for text files
loader = DirectoryLoader('documents/', glob="*.txt", loader_cls=TextLoader, loader_kwargs={'encoding': 'UTF-8'})
# Load documents
documents = loader.load()


In [4]:
documents[0]

Document(page_content='Titre: 10ÈME ÉDITION DU CONCOURS FRANCOPHONE INTERNATIONAL «\xa0MA THÈSE EN 180 SECONDES\xa0», Date: janvier 10, 2024. Pour plus de détails, veuillez consulter le lien suivant : https://fstt.ac.ma/Portail2023/10eme-edition-du-concours-francophone-international-ma-these-en-180-secondes/', metadata={'source': 'documents\\10ÈME_ÉDITION_DU_CONCOURS_FRANCOPHONE_INTERNATIONAL_MA_THÈSE_EN_180_SECONDES.txt'})

In [5]:
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_splitter = RecursiveCharacterTextSplitter()
chunks = text_splitter.split_documents(documents)

In [6]:
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True,model_kwargs={"device": "cuda"}),
    collection_name="txt_col"
)

OllamaEmbeddings: 100%|███████████████████| 385/385 [13:45<00:00,  2.14s/it]


In [7]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [8]:
# LLM from Ollama
local_model = "llama3"
llm = ChatOllama(model=local_model,language = "fr",model_kwargs={"device": "cuda"})

In [9]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Vous êtes un assistant modèle linguistique IA. Votre tâche consiste à générer deux
    versions différentes de la question utilisateur donnée pour récupérer les documents pertinents d'une
    base de données vectorielle. En générant plusieurs perspectives sur la question de l'utilisateur,
    votre objectif est d'aider l'utilisateur à surmonter certaines des limitations de la recherche par similarité basée sur la distance.
    Fournissez ces questions alternatives séparées par des nouvelles lignes.
    Question originale : {question}""",
)


In [10]:
retriever = MultiQueryRetriever.from_llm(vector_db.as_retriever(), llm,prompt=QUERY_PROMPT)
# RAG prompt

template = """Répondez à la question en français sur la base du contexte suivant :
{context}

Si la réponse à la question n'existe pas dans le contexte ci-dessus, veuillez indiquer que des informations supplémentaires sont nécessaires.

Question : {question}

"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [12]:
print(chain.invoke("qui est le coordinateur de MST: intelligence artificielle et sciences des données ?"))

OllamaEmbeddings: 100%|███████████████████████| 1/1 [00:02<00:00,  2.10s/it]


Selon le contexte, le coordinateur du Master en Sciences et Techniques (MST) : Sécurité IT et Big Data est le Pr.Abdelhamid ZOUHAIR. Son email est mstsit.bd@gmail.com.
